In [ ]:
import sys
sys.path.append("../../digitech_classify")

from digitech_classify.config import RAW_DATA_DIR, INTERIM_DATA_DIR
from digitech_classify.features import extract_noun_phrases, batch_lemmatize, nlp_fast, nlp_full
import pandas as pd




In [ ]:

companies_df = pd.read_csv(RAW_DATA_DIR / "cb_net0_companies_concat.csv",
        usecols=['org_ID', 'organisation_name', 'short_description', 'data_source'],
        dtype={'org_ID': 'string', 'organisation_name': 'string', 'short_description': 'string', 'data_source': 'string'},
        index_col=False)  
companies_df = companies_df[companies_df['data_source'] != 'Net0']

print(companies_df.shape)

In [ ]:
companies_df['search_text'] = (
    companies_df['short_description']
    .fillna('')
    .str.lower()
    .str.replace(r'[^\w\s]', ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

companies_df.drop(['short_description'], axis=1, inplace=True)

In [ ]:
companies_df['features'] = extract_noun_phrases(companies_df['search_text'].tolist(), model=nlp_full, batch_size=5000, n_process=2)


In [ ]:
save_path = INTERIM_DATA_DIR / "noun_phrase_companies.csv"
companies_df.to_csv(save_path, index=False)

In [ ]:
companies_df = pd.read_csv(INTERIM_DATA_DIR / "noun_phrase_companies.csv",
        usecols=['org_ID', 'organisation_name', 'features'])

In [ ]:
companies_df = companies_df.dropna(subset=['search_text'])

companies_df['features_lemma'] = batch_lemmatize(
    companies_df['search_text'].tolist(), model=nlp_fast, batch_size=5000
)




In [ ]:
companies_df = companies_df.drop(columns=['data_source', 'search_text'])

In [ ]:
save_path = INTERIM_DATA_DIR / "companies_lemma_short_description.csv"
companies_df.to_csv(save_path, index=False)